In [1]:
import pandas as pd
import numpy as np
from IPython.display import display as d

In [2]:
def max_display_columns(df: pd.core.frame.DataFrame):
    """Given a pandas df, display all columns"""
    with pd.option_context("display.max_columns", None):
        d(df)

## Transactions 

In [48]:
transactions = pd.read_parquet('credit_card_transaction_data_de.parquet')

In [49]:
# Clean the amounts field, remove $
transactions['Amount'] = transactions['Amount'].str.replace('$', '', regex=True).astype(float)

In [50]:
transactions = transactions.reset_index(drop=True)

In [55]:
transactions['DayOnlyTime'] = transactions['Time'].between('10:00', '20:00')

In [56]:
transactions.rename(columns={'Errors?':'Errors', 'Is Fraud?':'Fraud'}, inplace=True)

In [57]:
transaction_columns = ['User', 'Card', 'Amount', 'Use Chip', 'DayOnlyTime',
                       'Merchant State', 'MCC', 'Errors', 'Fraud']
transactions = transactions[transaction_columns]

### Users

In [8]:
users = pd.read_parquet('credit_card_users_de.parquet')

In [9]:
users_columns = ['Current Age', 'Retirement Age', 'State',  
                 'Per Capita Income - Zipcode', 'Yearly Income - Person', 
                 'Total Debt', 'FICO Score', 'Num Credit Cards']
users = users[users_columns]

In [10]:
users['CurrentlyRetired'] = users['Retirement Age'] <= users['Current Age'] 

In [11]:
# Convert some columns to float
for col in ['Per Capita Income - Zipcode', 'Yearly Income - Person', 'Total Debt']:
    users[col] = users[col].str.replace('$', '', regex=True).astype(float)

In [12]:
users['DebtIncomeRatio'] = users['Total Debt'].div(users['Yearly Income - Person'])
users['ZipIncomeRatio'] = users['Per Capita Income - Zipcode'].div(users['Yearly Income - Person'])

In [13]:
users.drop(columns=['Retirement Age', 'Total Debt', 'Per Capita Income - Zipcode'], inplace=True)

In [14]:
users = users.reset_index().rename(columns={'index':'User'})

### Cards

In [15]:
cards = pd.read_parquet('sd254_cards_de.parquet')

In [16]:
cards_columns = ['User', 'CARD INDEX', 'Card Brand', 'Card Type',
                 'Has Chip', 'Cards Issued', 'Credit Limit']
cards = cards[cards_columns]

In [17]:
cards['Credit Limit'] = cards['Credit Limit'].str.replace('$', '', regex=True).astype(float)

### Combine All

In [69]:
user_and_card = users.merge(cards,
                            how='left',
                            left_on='User',
                            right_on='User')
user_and_card

,User,Current Age,State,Yearly Income - Person,FICO Score,Num Credit Cards,CurrentlyRetired,DebtIncomeRatio,ZipIncomeRatio,CARD INDEX,Card Brand,Card Type,Has Chip,Cards Issued,Credit Limit
0,0,53,CA,59696.0,787,5,False,2.137714,0.490452,0,Visa,Debit,YES,2,24295.0
1,0,53,CA,59696.0,787,5,False,2.137714,0.490452,1,Visa,Debit,YES,2,21968.0
2,0,53,CA,59696.0,787,5,False,2.137714,0.490452,2,Visa,Debit,YES,2,46414.0
3,0,53,CA,59696.0,787,5,False,2.137714,0.490452,3,Visa,Credit,NO,1,12400.0
4,0,53,CA,59696.0,787,5,False,2.137714,0.490452,4,Mastercard,Debit (Prepaid),YES,1,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,1997,47,NJ,30942.0,779,3,False,2.296749,0.490434,1,Amex,Credit,YES,1,6900.0
6142,1997,47,NJ,30942.0,779,3,False,2.296749,0.490434,2,Visa,Credit,YES,2,5700.0
6143,1998,66,PA,54654.0,618,1,True,0.498426,0.463571,0,Mastercard,Credit,NO,2,9200.0
6144,1999,21,NH,65909.0,673,2,False,2.750171,0.490449,0,Mastercard,Debit,YES,1,28074.0


In [70]:
all_joined = transactions.merge(user_and_card,
                              left_on=['User', 'Card'],
                              right_on=['User', 'CARD INDEX'])

In [71]:
max_display_columns(all_joined)

,User,Card,Amount,Use Chip,DayOnlyTime,Merchant State,MCC,Errors,Fraud,Current Age,State,Yearly Income - Person,FICO Score,Num Credit Cards,CurrentlyRetired,DebtIncomeRatio,ZipIncomeRatio,CARD INDEX,Card Brand,Card Type,Has Chip,Cards Issued,Credit Limit
0,0,0,66.48,Chip Transaction,True,CA,7538,None,No,53,CA,59696.0,787,5,False,2.137714,0.490452,0,Visa,Debit,YES,2,24295.0
1,0,0,40.02,Chip Transaction,False,CA,5912,None,No,53,CA,59696.0,787,5,False,2.137714,0.490452,0,Visa,Debit,YES,2,24295.0
2,0,0,54.11,Chip Transaction,False,CA,7538,None,No,53,CA,59696.0,787,5,False,2.137714,0.490452,0,Visa,Debit,YES,2,24295.0
3,0,0,89.48,Chip Transaction,True,CA,5651,None,No,53,CA,59696.0,787,5,False,2.137714,0.490452,0,Visa,Debit,YES,2,24295.0
4,0,0,29.15,Chip Transaction,False,CA,5942,None,No,53,CA,59696.0,787,5,False,2.137714,0.490452,0,Visa,Debit,YES,2,24295.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877832,1999,1,66.53,Chip Transaction,True,NH,7538,None,No,21,NH,65909.0,673,2,False,2.750171,0.490449,1,Visa,Credit,YES,1,14400.0
6877833,1999,1,99.67,Chip Transaction,True,NH,5411,None,No,21,NH,65909.0,673,2,False,2.750171,0.490449,1,Visa,Credit,YES,1,14400.0
6877834,1999,1,93.63,Chip Transaction,True,NH,5651,None,No,21,NH,65909.0,673,2,False,2.750171,0.490449,1,Visa,Credit,YES,1,14400.0
6877835,1999,1,67.60,Online Transaction,False,None,4121,None,No,21,NH,65909.0,673,2,False,2.750171,0.490449,1,Visa,Credit,YES,1,14400.0


In [72]:
# Columns based on all transactions
all_joined['MerchantCustomerStateMatch'] = (all_joined['State'] == all_joined['Merchant State'])
all_joined['InternationalTransaction'] = all_joined['Merchant State'].isin(all_joined['State'].unique())

In [73]:
all_joined = all_joined[['Card', 'Amount', 'Use Chip', 'Merchant State', 'MCC', 'Errors',
       'Current Age', 'Yearly Income - Person', 'FICO Score', 'DayOnlyTime',
       'Num Credit Cards', 'CurrentlyRetired', 'DebtIncomeRatio',
       'ZipIncomeRatio', 'Card Brand', 'Card Type', 'Has Chip',
       'Cards Issued', 'Credit Limit', 'MerchantCustomerStateMatch', 
        'InternationalTransaction', 'Fraud']]

In [68]:
all_joined.to_parquet('joined_transactions.parquet')